### Deliverable 3: Optimization 
Attempt 1: Remove noisy variables from features

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os 
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Remove possibly noisy variables
application_df = application_df.drop(['STATUS', 'SPECIAL_CONSIDERATIONS','CLASSIFICATION'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,0,5000,1
1,T3,Independent,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...
34294,T4,Independent,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,Preservation,Association,0,5000,0
34297,T5,Independent,ProductDev,Association,0,5000,1


In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df.APPLICATION_TYPE.value_counts()
#application_type_counts

In [6]:
# Visualize the value counts of APPLICATION_TYPE
#application_type_counts.plot.density()

In [7]:
# Determine which values to replace if counts are less than ...?
replace_application_types = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application_types:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
#classification_counts = application_df.CLASSIFICATION.value_counts()
#classification_counts

In [9]:
# Visualize the value counts of CLASSIFICATION
#classification_counts.plot.density()

In [10]:
# Determine which values to replace if counts are less than ..?
#replace_classification = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
#for cls in replace_classification:
    #application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
#application_df.CLASSIFICATION.value_counts()

In [11]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
#encode_df.head()

In [13]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)

In [14]:
application_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34296,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

#number_input_features

In [18]:
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                2800      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create callback
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=250)

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
242/804 [========>.....................] - ETA: 1s - loss: 0.6601 - accuracy: 0.6441
Epoch 00001: saving model to checkpoints\weights.01.hdf5
487/804 [=================>............] - ETA: 1s - loss: 0.6287 - accuracy: 0.6727
Epoch 00001: saving model to checkpoints\weights.01.hdf5
742/804 [==========================>...] - ETA: 0s - loss: 0.6156 - accuracy: 0.6849 ETA: 0s - los
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 5s 4ms/step - loss: 0.6133 - accuracy: 0.6870
Epoch 2/50
193/804 [======>.......................] - ETA: 2s - loss: 0.5607 - accuracy: 0.7399
Epoch 00002: saving model to checkpoints\weights.02.hdf5
435/804 [===============>..............] - ETA: 1s - loss: 0.5637 - accuracy: 0.7340
Epoch 00002: saving model to checkpoints\weights.02.hdf5
686/804 [========================>.....] - ETA: 0s - loss: 0.5659 - accuracy: 0.7311
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============

Epoch 00015: saving model to checkpoints\weights.15.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5596 - accuracy: 0.7323
Epoch 16/50
188/804 [======>.......................] - ETA: 1s - loss: 0.5592 - accuracy: 0.7360
Epoch 00016: saving model to checkpoints\weights.16.hdf5
414/804 [==============>...............] - ETA: 1s - loss: 0.5616 - accuracy: 0.7333
Epoch 00016: saving model to checkpoints\weights.16.hdf5
675/804 [========================>.....] - ETA: 0s - loss: 0.5619 - accuracy: 0.7324
Epoch 00016: saving model to checkpoints\weights.16.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5619 - accuracy: 0.7321
Epoch 17/50
115/804 [===>..........................] - ETA: 1s - loss: 0.5504 - accuracy: 0.7371
Epoch 00017: saving model to checkpoints\weights.17.hdf5
379/804 [=============>................] - ETA: 0s - loss: 0.5568 - accuracy: 0.7334
Epoch 00017: saving model to checkpoints\weights.17.hdf5
618/804 [======================

418/804 [==============>...............] - ETA: 1s - loss: 0.5670 - accuracy: 0.7227
Epoch 00030: saving model to checkpoints\weights.30.hdf5
682/804 [========================>.....] - ETA: 0s - loss: 0.5661 - accuracy: 0.7246
Epoch 00030: saving model to checkpoints\weights.30.hdf5
804/804 [==============================] - 4s 4ms/step - loss: 0.5655 - accuracy: 0.7254
Epoch 31/50
117/804 [===>..........................] - ETA: 2s - loss: 0.5780 - accuracy: 0.7139
Epoch 00031: saving model to checkpoints\weights.31.hdf5
372/804 [============>.................] - ETA: 1s - loss: 0.5736 - accuracy: 0.7188
Epoch 00031: saving model to checkpoints\weights.31.hdf5
622/804 [======================>.......] - ETA: 0s - loss: 0.5694 - accuracy: 0.7227
Epoch 00031: saving model to checkpoints\weights.31.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5678 - accuracy: 0.7243
Epoch 32/50
 72/804 [=>............................] - ETA: 2s - loss: 0.5513 - accuracy: 0.7388
Epo

804/804 [==============================] - 2s 2ms/step - loss: 0.5619 - accuracy: 0.7318
Epoch 45/50
111/804 [===>..........................] - ETA: 1s - loss: 0.5610 - accuracy: 0.7240
Epoch 00045: saving model to checkpoints\weights.45.hdf5
368/804 [============>.................] - ETA: 1s - loss: 0.5620 - accuracy: 0.7269
Epoch 00045: saving model to checkpoints\weights.45.hdf5
621/804 [======================>.......] - ETA: 0s - loss: 0.5618 - accuracy: 0.7279
Epoch 00045: saving model to checkpoints\weights.45.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5619 - accuracy: 0.7283
Epoch 46/50
 59/804 [=>............................] - ETA: 1s - loss: 0.5522 - accuracy: 0.7344
Epoch 00046: saving model to checkpoints\weights.46.hdf5
316/804 [==========>...................] - ETA: 1s - loss: 0.5601 - accuracy: 0.7303
Epoch 00046: saving model to checkpoints\weights.46.hdf5
566/804 [====================>.........] - ETA: 0s - loss: 0.5616 - accuracy: 0.7297
Epo

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5705 - accuracy: 0.7244
Loss: 0.5705162882804871, Accuracy: 0.7244315147399902


In [22]:
# Export your results to an HDF5 file
nn.save('AlphabetSoupCharity_attempt1.h5')